## ML Preprocessing
### DSCI 525 - Group 10


In [21]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

### Data prep

We are going to read in multiple weather model forecasts for Australia from parquet files, trim to Sydney forecasts and then join on actual observed rainfall data. We will aggregate to *mean* daily rainfall from the models to compare to actuals. The final data frame will have a separate column for each weather model to be used for machine learning modelling of the actual rainfall in Sydney.

In [22]:
# INPUTS ------------------------------------------
# Due to difficulties with permissions on S3 buckets, we've been using a few iterations here
path_to_input_datasets = "/srv/data/my_shared_data_folder/"
# path_to_input_datasets = "s3://mds-s3-student71/"

path_to_output = "/srv/data/my_shared_data_folder/"
# path_to_output = "s3://mds-s3-student71/output/"

# CONSTANTS ----------------------------------------
syd_lat = -33.86
syd_lon = 151.21

df_actuals = (pd.read_csv(os.path.join(path_to_input_datasets, "observed_daily_rainfall_SYD.csv"),
                          names=["time","observed_rainfall"],
                          header=0,
                          parse_dates=True,
                          index_col="time"
                         ))

# Get all weather model data, aggregate to daily rainfall numbers by taking mean, 
# and join on observed values read in above.
df_modelling = (pd.read_parquet(os.path.join(path_to_input_datasets, "combined_model_data_parti.parquet"))
      .query(f"lat_min < {syd_lat} and lat_max > {syd_lat} and lon_min < {syd_lon} and lon_max > {syd_lon}")\
    [["time", "model", "rain (mm/day)"]]
      .assign(time = lambda x:x["time"].dt.date)
      .pivot_table(index="time", columns="model", values="rain (mm/day)")
      .merge(df_actuals[["observed_rainfall"]], left_index=True, right_index=True)
     )

df_modelling.head()

,ACCESS-CM2,ACCESS-ESM1-5,AWI-ESM-1-1-LR,BCC-CSM2-MR,BCC-ESM1,CMCC-CM2-HR4,CMCC-CM2-SR5,CMCC-ESM2,CanESM5,EC-Earth3-Veg-LR,...,MPI-ESM-1-2-HAM,MPI-ESM1-2-HR,MPI-ESM1-2-LR,MRI-ESM2-0,NESM3,NorESM2-LM,NorESM2-MM,SAM0-UNICON,TaiESM1,observed_rainfall
time,,,,,,,,,,,,,,,,,,,,,
1889-01-01,0.040427,1.814552,35.579336,4.268112e+00,1.107466e-03,11.410537,3.322009e-08,2.668800,1.321215,1.515293,...,4.244226e-13,1.390174e-13,6.537884e-05,3.445495e-06,1.576096e+01,4.759651e-05,2.451075,0.221324,2.257933,0.006612
1889-01-02,0.073777,0.303965,4.596520,1.190141e+00,1.015323e-04,4.014984,1.312700e+00,0.946211,2.788724,4.771375,...,4.409552e+00,1.222283e-01,1.049131e-13,4.791993e-09,3.675510e-01,4.350863e-01,0.477231,3.757179,2.287381,0.090422
1889-01-03,0.232656,0.019976,5.927467,1.003845e-09,1.760345e-05,9.660565,9.103720e+00,0.431999,0.003672,4.233980,...,2.269300e-01,3.762301e-01,9.758706e-14,6.912302e-01,1.562869e-01,9.561101e+00,0.023083,0.253357,1.199909,1.401452
1889-01-04,0.911319,13.623777,8.029624,8.225225e-02,1.808932e-01,3.951528,1.317160e+01,0.368693,0.013578,15.252495,...,2.344586e-02,4.214019e-01,7.060915e-03,3.835721e-02,2.472226e-07,5.301038e-01,0.002699,2.185454,2.106737,14.869798
1889-01-05,0.698013,0.021048,2.132686,2.496841e+00,4.708019e-09,2.766362,1.822940e+01,0.339267,0.002468,11.920356,...,4.270161e-13,1.879692e-01,4.504985e+00,3.506923e-07,1.949792e-13,1.460928e-10,0.001026,2.766507,1.763335,0.467628


We write to `S3` for object persistence and downstream modelling.

In [23]:
df_modelling.to_csv(os.path.join(path_to_output, "ml_data_SYD.csv"))